# Ensemble methods

## Computational Machine Learning assignment - Alessandro Tenderini


First, I import the necessary modules 

In [148]:
import matplotlib.pylab as plt
import sys
sys.path.insert(1, '/Users/utente/desktop/BARCELLONA/TERM 1/Computational Machine Learning/CML_materials')
from utils.helper_functions import *

import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.preprocessing import OneHotEncoder       
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from sklearn import preprocessing                    
from sklearn.svm import SVR                           
from sklearn.model_selection import GridSearchCV     
import dateutil.parser
import category_encoders as ce                         
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer             
from sklearn.neighbors import KNeighborsRegressor     
from sklearn.metrics import r2_score, mean_squared_error, make_scorer
from sklearn.model_selection import RepeatedKFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.impute import KNNImputer


from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import ColumnTransformer
from sklearn import set_config
from category_encoders import LeaveOneOutEncoder
from imblearn.pipeline import Pipeline as Pipeline
from category_encoders import WOEEncoder
from sklearn.experimental import enable_halving_search_cv 
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.ensemble import ExtraTreesRegressor, StackingRegressor
from sklearn.linear_model import LinearRegression

import mlens
from mlens.visualization import corrmat
from mlens.ensemble import SuperLearner

from sklearn.neural_network import MLPRegressor

## Data preprocessing

I import the train and test datataset. I also import an extra dataset which contains useful information about the diagnosis of each combination of Patient identifier (subject_id) and Hospital stay identifier (hadm_id).


* I remove 'Diff' as I do not need realistic datetimes.

* I remove 'DIAGNOSIS' as I already have the corresponding ICD9_CODE. 

* I remove 'HOSPITAL_EXPIRE_FLAG' as we are not supposed to know this variable.

In [149]:
data = pd.read_csv("/Users/utente/desktop/BARCELLONA/TERM 1/Computational Machine Learning/CML_materials/extended 2/data/mimic_train.csv") 
data= data.drop(["Diff","HOSPITAL_EXPIRE_FLAG","DIAGNOSIS"], axis=1)  #"DOD","DISCHTIME","DEATHTIME","Diff","DIAGNOSIS"

X_test = pd.read_csv("/Users/utente/desktop/BARCELLONA/TERM 1/Computational Machine Learning/CML_materials/extended 2/data/mimic_test_los.csv") 
X_test=X_test.drop(["Diff","DIAGNOSIS"], axis=1)

y_train = data["LOS"]   

#each subject (subject_id) has different diagnosis (ICD9_CODE)
ICD9_code= pd.read_csv("/Users/utente/desktop/BARCELLONA/TERM 1/Computational Machine Learning/CML_materials/extended 2/data/MIMIC_diagnoses.csv") 
ICD9_code.columns=['subject_id', 'hadm_id', 'SEQ_NUM', 'ICD9_CODE']

We can include more about diagnosis as in ICD9_code there are all the diagnosis of each combination of HADM_ID and SUBJECT_ID. Hence, I can extract a list of diagnosis for each icustay_id in TEST and TRAIN according to the combination of HADM_ID and SUBJECT_ID.

### Target encoder and diagnosis dataset 

* I create a temporary dataset called "tepm", which is an INNER JOIN on the combination of 'subject_id' and 'hadm_id' of original data set and the extra diagnosis datatset.

* Now, I perform a target encoding on the column containig the different diagnosis as ICD9_CODE. Then, for each diagnosis, I get the portion of people who died among those who had that diagnosis.

* I create three new variables for each combination of 'subject_id' and 'hadm_id', based on the previously computed portion of death people. Such variables are the max value between the portions of death people for each diagnosis of a single combination, the mean of these, and the number of diagnosis of each combination.


I do the same procedure for the test set with a temporary dataset called "tepm2": I transform the dataset based on the target encoder trained on the training set and follow the same steps.




In [150]:
temp=pd.merge(data, ICD9_code, on=['subject_id','hadm_id'], how='inner')
y_train = temp["LOS"]   
temp= temp.drop(['LOS'], axis=1)  #"DOD","DISCHTIME","DEATHTIME","LOS","Diff","DIAGNOSIS"
temp2=pd.merge(X_test, ICD9_code, on=['subject_id','hadm_id'], how='inner')
enc = ce.TargetEncoder(cols=['ICD9_CODE'], min_samples_leaf=20, smoothing=10).fit(temp, y_train)
temp = enc.transform(temp)
temp2 = enc.transform(temp2)

data=pd.merge(data, ICD9_code.groupby(['subject_id','hadm_id'],as_index=False)['SEQ_NUM'].count(), on=['subject_id','hadm_id'])
data=pd.merge(data, temp.groupby(['subject_id','hadm_id'],as_index=False)['ICD9_CODE'].mean('ICD9_CODE'), on=['subject_id','hadm_id'])
data=pd.merge(data, temp.groupby(['subject_id','hadm_id'],as_index=False)['ICD9_CODE'].max('ICD9_CODE'), on=['subject_id','hadm_id'])

X_test=pd.merge(X_test, ICD9_code.groupby(['subject_id','hadm_id'],as_index=False)['SEQ_NUM'].count(), on=['subject_id','hadm_id'])
X_test=pd.merge(X_test, temp2.groupby(['subject_id','hadm_id'],as_index=False)['ICD9_CODE'].mean('ICD9_CODE'), on=['subject_id','hadm_id'])
X_test=pd.merge(X_test, temp2.groupby(['subject_id','hadm_id'],as_index=False)['ICD9_CODE'].max('ICD9_CODE'), on=['subject_id','hadm_id'])



* These are the y_train (lenght of stay) and X_train datatset I start with. I also save the icustay_id of both the train and test set



In [151]:

y_train = data["LOS"]   
X_train=data.drop(["LOS"], axis=1)

icustay_id_test=X_test["icustay_id"]
icustay_id_train=X_train["icustay_id"]  

### Age

I calculate the age of the patients based on the date of birth (DOB) and the amdission date (ADMITTIME) and I subsequently remove the variable 'DOB' and 'ADMITTIME'.

In [152]:
X_train['Age'] = np.nan
X_test['Age'] = np.nan

for index in range(len(X_train)):
    start = dateutil.parser.parse(X_train['DOB'][index])
    end = dateutil.parser.parse(X_train['ADMITTIME'][index])
    X_train['Age'][index]= round((end-start).days/365)

for index in range(len(X_test)):
    start = dateutil.parser.parse(X_test['DOB'][index])
    end = dateutil.parser.parse(X_test['ADMITTIME'][index])
    X_test['Age'][index]= round((end-start).days/365)
    
    
X_test=X_test.drop(['DOB'], axis=1)  
X_train=X_train.drop(['DOB'], axis=1) 

X_train.loc[X_train['Age'] >=90, 'Age'] = 90
X_test.loc[X_test['Age'] >=90, 'Age'] = 90

/var/folders/y1/d2rn46357hl_mm2r9x_598q80000gn/T/ipykernel_12480/2339022325.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['Age'][index]= round((end-start).days/365)
/var/folders/y1/d2rn46357hl_mm2r9x_598q80000gn/T/ipykernel_12480/2339022325.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['Age'][index]= round((end-start).days/365)


###  Ethnicity 

I combine some levels of ethnicity to reduce the cardinality of the variable

In [153]:

# A) Combing Ethnicity (train set)
X_train['ETHNICITY'] = X_train['ETHNICITY'].replace(['ASIAN', 'ASIAN - CHINESE', 'ASIAN - ASIAN INDIAN', 'ASIAN - VIETNAMESE', 'ASIAN - FILIPINO', 'ASIAN - CAMBODIAN',
                                                     'ASIAN - JAPANESE', 'ASIAN - THAI', 'ASIAN - OTHER', 'ASIAN - KOREAN'
                                                     ], 'ASIAN')

X_train['ETHNICITY'] = X_train['ETHNICITY'].replace(['HISPANIC 0R LATINO', 'HISPANIC/LATINO - PUERTO RICAN', 'HISPANIC/LATINO - DOMINICAN',
                                                     'HISPANIC/LATINO - GUATEMALAN', 'HISPANIC/LATINO - CUBAN', 'HISPANIC/LATINO - SALVADORAN',
                                                     'HISPANIC/LATINO - MEXICAN', 'HISPANIC/LATINO - CENTRAL AMERICAN (OTHER)', 'HISPANIC/LATINO - COLOMBIAN',
                                                     'HISPANIC/LATINO - HONDURAN', 'SOUTH AMERICAN'
                                                     ], 'HISPANIC OR LATINO')

X_train['ETHNICITY'] = X_train['ETHNICITY'].replace(['WHITE', 'WHITE - RUSSIAN', 'WHITE - OTHER EUROPEAN', 'WHITE - EASTERN EUROPEAN',
                                                     'WHITE - BRAZILIAN'
                                                     ], 'WHITE')

X_train['ETHNICITY'] = X_train['ETHNICITY'].replace(['BLACK/AFRICAN', 'BLACK/AFRICAN AMERICAN', 'BLACK/CAPE VERDEAN', 'BLACK/HAITIAN'
                                                     ], 'BLACK')

X_train['ETHNICITY'] = X_train['ETHNICITY'].replace(['UNABLE TO OBTAIN', 'UNKNOWN/NOT SPECIFIED', 'PATIENT DECLINED TO ANSWER'
                                                     ], 'UNKNOWN')

X_train['ETHNICITY'] = X_train['ETHNICITY'].replace(['AMERICAN INDIAN/ALASKA NATIVE', 'AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE',
                                                     'CARIBBEAN ISLAND', 'MIDDLE EASTERN', 'OTHER', 'PORTUGUESE', 'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER',
                                                     'MULTI RACE ethnicity'
                                                      ], 'OTHER')

# B) Combing Ethnicity (test set)
X_test['ETHNICITY'] = X_test['ETHNICITY'].replace(['ASIAN', 'ASIAN - CHINESE', 'ASIAN - ASIAN INDIAN', 'ASIAN - VIETNAMESE', 'ASIAN - FILIPINO', 'ASIAN - CAMBODIAN',
                                                     'ASIAN - JAPANESE', 'ASIAN - THAI', 'ASIAN - OTHER', 'ASIAN - KOREAN'
                                                     ], 'ASIAN')

X_test['ETHNICITY'] = X_test['ETHNICITY'].replace(['HISPANIC 0R LATINO', 'HISPANIC/LATINO - PUERTO RICAN', 'HISPANIC/LATINO - DOMINICAN',
                                                     'HISPANIC/LATINO - GUATEMALAN', 'HISPANIC/LATINO - CUBAN', 'HISPANIC/LATINO - SALVADORAN',
                                                     'HISPANIC/LATINO - MEXICAN', 'HISPANIC/LATINO - CENTRAL AMERICAN (OTHER)', 'HISPANIC/LATINO - COLOMBIAN',
                                                     'HISPANIC/LATINO - HONDURAN', 'SOUTH AMERICAN'
                                                     ], 'HISPANIC OR LATINO')

X_test['ETHNICITY'] = X_test['ETHNICITY'].replace(['WHITE', 'WHITE - RUSSIAN', 'WHITE - OTHER EUROPEAN', 'WHITE - EASTERN EUROPEAN',
                                                     'WHITE - BRAZILIAN'
                                                     ], 'WHITE')

X_test['ETHNICITY'] = X_test['ETHNICITY'].replace(['BLACK/AFRICAN', 'BLACK/AFRICAN AMERICAN', 'BLACK/CAPE VERDEAN', 'BLACK/HAITIAN'
                                                     ], 'BLACK')

X_test['ETHNICITY'] = X_test['ETHNICITY'].replace(['UNABLE TO OBTAIN', 'UNKNOWN/NOT SPECIFIED', 'PATIENT DECLINED TO ANSWER'
                                                     ], 'UNKNOWN')

X_test['ETHNICITY'] = X_test['ETHNICITY'].replace(['AMERICAN INDIAN/ALASKA NATIVE', 'AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE',
                                                     'CARIBBEAN ISLAND', 'MIDDLE EASTERN', 'OTHER', 'PORTUGUESE', 'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER',
                                                     'MULTI RACE ethnicity'
                                                      ], 'OTHER')

# Count values for features per category
X_train['ETHNICITY'].value_counts()
X_test['ETHNICITY'].value_counts()

WHITE                   3849
BLACK                    532
UNKNOWN                  352
HISPANIC OR LATINO       203
ASIAN                    144
OTHER                    129
MULTI RACE ETHNICITY      12
Name: ETHNICITY, dtype: int64

### Religion

I do the same for religion

In [154]:

religion_other = ['HEBREW', 'UNITARIAN-UNIVERSALIST', 'HINDU', 'GREEK ORTHODOX',"JEHOVAH'S WITNESS", "BUDDHIST", 'MUSLIM', 'OTHER', 'CHRISTIAN SCIENTIST', 'EPISCOPALIAN', 'ROMANIAN EAST. ORTH', '7TH DAY ADVENTIST']
X_train['RELIGION'] = X_train['RELIGION'].replace(religion_other, 'OTHER')



X_test['RELIGION'] = X_test['RELIGION'].replace(religion_other, 'OTHER')


X_train['RELIGION'].value_counts()
X_test['RELIGION'].value_counts()



CATHOLIC             1898
NOT SPECIFIED        1361
PROTESTANT QUAKER     697
JEWISH                446
OTHER                 424
UNOBTAINABLE          395
Name: RELIGION, dtype: int64

### Repeated visits to ICU

I add a variable that defines the number of the visits for a patient (for patients who went in ICU more tha once)

In [155]:
X_train["visits_ICU"] = X_train.sort_values(['subject_id', 'ADMITTIME']).groupby(["subject_id"]).cumcount() + 1
X_test["visits_ICU"] = X_test.sort_values(['subject_id', 'ADMITTIME']).groupby(["subject_id"]).cumcount() + 1


X_train[["subject_id", "ADMITTIME", "visits_ICU"]].sort_values(["subject_id", "ADMITTIME"]).iloc[:13]

# Eliminate ADMITTIME
X_train = X_train.drop(['ADMITTIME'], axis = 1)
X_test = X_test.drop(['ADMITTIME'], axis = 1)

I "no category" for Na in MARITAL_STATUS

In [156]:
X_train['MARITAL_STATUS'] = X_train['MARITAL_STATUS'].fillna('no category')
X_test['MARITAL_STATUS'] = X_test['MARITAL_STATUS'].fillna('no category')

I run Foward and Backweard filling with previous visit information

In [157]:
before=X_train.isnull().sum()
X_train = X_train.groupby(['subject_id'], as_index = False).apply(lambda group: group.ffill())
X_train = X_train.groupby(['subject_id'], as_index = False).apply(lambda group: group.bfill())
after=X_train.isnull().sum()

/var/folders/y1/d2rn46357hl_mm2r9x_598q80000gn/T/ipykernel_12480/748160573.py:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  X_train = X_train.groupby(['subject_id'], as_index = False).apply(lambda group: group.ffill())
/var/folders/y1/d2rn46357hl_mm2r9x_598q80000gn/T/ipykernel_12480/748160573.py:3: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this 

These are the amount of missing values that were filled

In [158]:
before-after ##Really nice

subject_id          0
hadm_id             0
icustay_id          0
HeartRate_Min     400
HeartRate_Max     400
HeartRate_Mean    400
SysBP_Min         404
SysBP_Max         404
SysBP_Mean        404
DiasBP_Min        404
DiasBP_Max        404
DiasBP_Mean       404
MeanBP_Min        398
MeanBP_Max        398
MeanBP_Mean       398
RespRate_Min      402
RespRate_Max      402
RespRate_Mean     402
TempC_Min         469
TempC_Max         469
TempC_Mean        469
SpO2_Min          403
SpO2_Max          403
SpO2_Mean         403
Glucose_Min        62
Glucose_Max        62
Glucose_Mean       62
GENDER              0
ADMISSION_TYPE      0
INSURANCE           0
RELIGION            0
MARITAL_STATUS      0
ETHNICITY           0
ICD9_diagnosis      0
FIRST_CAREUNIT      0
SEQ_NUM             0
ICD9_CODE_x         0
ICD9_CODE_y         0
Age                 0
visits_ICU          0
dtype: int64

I do the same for the test set

In [159]:

X_test = X_test.groupby(['subject_id'], as_index = False).apply(lambda group: group.ffill())
X_test = X_test.groupby(['subject_id'], as_index = False).apply(lambda group: group.bfill())

/var/folders/y1/d2rn46357hl_mm2r9x_598q80000gn/T/ipykernel_12480/645426860.py:1: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  X_test = X_test.groupby(['subject_id'], as_index = False).apply(lambda group: group.ffill())
/var/folders/y1/d2rn46357hl_mm2r9x_598q80000gn/T/ipykernel_12480/645426860.py:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this wa

I save the 3 id of the train and test set and drop them from the two dataset

In [160]:

Id_train=X_train[["subject_id", "hadm_id","icustay_id"]]
X_train=X_train.drop(["subject_id", "hadm_id","icustay_id"], axis=1)
Id_test=X_test[["subject_id", "hadm_id","icustay_id"]]
X_test=X_test.drop(["subject_id", "hadm_id","icustay_id"], axis=1)

This is the situation with missing values. I am going to imput them in the pipeline.

In [161]:
X_train.isnull().sum()

HeartRate_Min     1787
HeartRate_Max     1787
HeartRate_Mean    1787
SysBP_Min         1804
SysBP_Max         1804
SysBP_Mean        1804
DiasBP_Min        1805
DiasBP_Max        1805
DiasBP_Mean       1805
MeanBP_Min        1788
MeanBP_Max        1788
MeanBP_Mean       1788
RespRate_Min      1787
RespRate_Max      1787
RespRate_Mean     1787
TempC_Min         2028
TempC_Max         2028
TempC_Mean        2028
SpO2_Min          1800
SpO2_Max          1800
SpO2_Mean         1800
Glucose_Min        191
Glucose_Max        191
Glucose_Mean       191
GENDER               0
ADMISSION_TYPE       0
INSURANCE            0
RELIGION             0
MARITAL_STATUS       0
ETHNICITY            0
ICD9_diagnosis       0
FIRST_CAREUNIT       0
SEQ_NUM              0
ICD9_CODE_x          0
ICD9_CODE_y          0
Age                  0
visits_ICU           0
dtype: int64

In [162]:
X_test.isnull().sum()

HeartRate_Min     507
HeartRate_Max     507
HeartRate_Mean    507
SysBP_Min         513
SysBP_Max         513
SysBP_Mean        513
DiasBP_Min        514
DiasBP_Max        514
DiasBP_Mean       514
MeanBP_Min        509
MeanBP_Max        509
MeanBP_Mean       509
RespRate_Min      508
RespRate_Max      508
RespRate_Mean     508
TempC_Min         592
TempC_Max         592
TempC_Mean        592
SpO2_Min          512
SpO2_Max          512
SpO2_Mean         512
Glucose_Min        52
Glucose_Max        52
Glucose_Mean       52
GENDER              0
ADMISSION_TYPE      0
INSURANCE           0
RELIGION            0
MARITAL_STATUS      0
ETHNICITY           0
ICD9_diagnosis      0
FIRST_CAREUNIT      0
SEQ_NUM             0
ICD9_CODE_x         0
ICD9_CODE_y         0
Age                 0
visits_ICU          0
dtype: int64

As we can see GENDER ,ADMISSION_TYPE, INSURANCE, RELIGION, MARITAL_STATUS, ETHNICITY, ICD9_diagnosis, FIRST_CAREUNIT, SEQ_NUM, ICD9_CODE_x, ICD9_CODE_y, Age and visits_ICU have no missing values.

For the other variables, I am going to imput their missing values in the preprocessing part of the pipeline.
       

## Preprocessing for the pipeline


First, I select the numerical and categorical variables. I also eliminate ICD9_diagnosis from categorical variables as, for this variable, I am going to imput NA in slightly different way.

In [163]:
num_feat = X_train.select_dtypes(exclude=['object', 'category']).columns
cat_feat = X_train.select_dtypes(include=['object', 'category']).columns
icd9_feat = ['ICD9_diagnosis']
cat_feat = cat_feat.drop(['ICD9_diagnosis'])



I am going to impute missing values in numerical variables with KNNImputer.

In [164]:
cont_imputer = KNNImputer()

I am going to impute missing values in categorical variables with LeaveOneOutEncoder.

In [165]:
cat_encoder = ce.LeaveOneOutEncoder()

I am going to impute missing values for ICD9, which is the main diagnosis with TargetEncoder. Also, I tried to do the same with the extra dataset but it was too complicated. Therefore, I target encoded the diagnosis from the extra dataset outside the pipeline. 

In [166]:
icd9_encoder = ce.TargetEncoder(smoothing = 1.0)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/category_encoders/target_encoder.py:122: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/category_encoders/target_encoder.py:127: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."


Last, I will use RobustScaler for numerical variables and StandardScaler for categorical variables.

In [167]:
cont_scaler = RobustScaler()
cat_scaler = StandardScaler()

This is the preprocessing pipeline for numerical variables.

In [168]:
num_preprocessing = make_pipeline(
    cont_scaler,
    cont_imputer
    )

This is the one for categorical variables.

In [169]:
cat_preprocessing = make_pipeline(
    cat_encoder,
    cat_scaler)

And this is the one only for ICD9.

In [170]:
icd9_preprocessing = make_pipeline(
    icd9_encoder,
    cat_scaler)

Finally, I use ColumnTransformer to merge togther the tree pipeline. Indeed, I am able to apply each preprocessing to the corresponding columns.   

In [171]:

preprocessing = ColumnTransformer(
    [("num", num_preprocessing, num_feat),
     ("cat", cat_preprocessing, cat_feat),
     ("icd9", icd9_preprocessing, icd9_feat)
     ]
    , remainder='drop'
)

## Ensemble methods


### Stackining 

I chose Stacking over bagging and boosting for two reasons. 

* First, it allows to use heterogeneous weak learners; combining different algorithms could potentially provide better results (also due to the different nature of the algorithms in terms of variance and bias). 

* Second, stacking combines the weak learners using a meta-model. On the other hand, bagging and boosting combine the base models according deterministic algorithms. Hence, the first model include a further level of analysis.



#### Weak learners 

These are the base models that I selected: Support vector machines, K-nearest neighbors, Linear regression and Decision tree.

In [172]:
svr = SVR() 
knn = KNeighborsRegressor()
lr = LinearRegression()
dt = DecisionTreeRegressor()

Before creating the stacking ensemble model, I have to chose the meta-model to make the final prediction based on combined predictions of the base models.

I am going to run two stack models with two different meta-models:

* ExtraTreesRegressor

* MLPRegressor (multi-layer perceptron model) which is a type of neural network


#### ExtraTreesRegressor ensemble model


I create the grid of parameters for hyperpameter optimization of the base models and the final estimator. 

In [173]:
param_grid1 = {'ensemble__svr__C': [0.1, 1, 10],
              'ensemble__svr__gamma': [0.1, 0.25, 0.5, 0.75],
              'ensemble__svr__kernel': ['linear', 'rbf', 'poly'],
              'ensemble__knn__n_neighbors': [5, 10, 20, 50, 100, 150],
              'ensemble__dt__max_depth':[25, 50, 75, 100],
              'ensemble__final_estimator__max_depth': [5,25, 50, 75],
              'ensemble__final_estimator__min_samples_split': [2, 4, 6],
              'ensemble__final_estimator__min_samples_leaf': [1, 2, 4]
              }              

I create ensemble model with the previously selected weak learners and ExtraTreesRegressor as final estimator. 

In [174]:
ensemble1 =StackingRegressor(estimators=[('svr', svr), ('knn', knn), ('lr', lr), ('dt', dt)], final_estimator=ExtraTreesRegressor())


I am also creating the pipeline with the preprocessing and then then the actual model.

In [175]:
pipeline1 = Pipeline([('preprocess', preprocessing),('ensemble', ensemble1)])

Finally, I run the grid search (HalvingGridSearchCV to reduce time) and find the best model according to root mean square error (RMSE).

In [176]:
grid_search1 = HalvingGridSearchCV(pipeline1, param_grid1, cv=5, n_jobs=-1, verbose=3,scoring ='neg_root_mean_squared_error')
#grid_search1.fit(X_train, y_train)

These are predictions and RMSE of the best model and the corresponding parameters.

In [177]:
#prediction_ET=grid_search1.predict(X_test)
#grid_search1.best_params_
#grid_search1.best_score_

#### Neural Network ensemble model

I do the same for Neural Network as final estimator.


In [178]:
NN = MLPRegressor()

ensemble2 =StackingRegressor(estimators=[('svr', svr), ('knn', knn), ('lr', lr),  ('dt', dt)], final_estimator=NN)
pipeline2 = Pipeline([('preprocess', preprocessing),('ensemble', ensemble2)])

I change the parameters to allow for hyperpameter optimization for some parameters of the neural network

In [179]:
param_grid2 = {'ensemble__svr__C': [0.1, 1, 10],
              'ensemble__svr__gamma': [0.1, 0.25, 0.5, 0.75],
              'ensemble__svr__kernel': ['linear', 'rbf', 'poly'],
              'ensemble__knn__n_neighbors': [5, 10, 20, 50, 100, 150],
              'ensemble__dt__max_depth':[25, 50, 75, 100],
              'ensemble__final_estimator__activation': ['tanh', 'relu'],
              'ensemble__final_estimator__hidden_layer_sizes': [(30,20,10,5), (30,20,10), (20,10,5), (10,5)],
              'ensemble__final_estimator__alpha': [0.0001, 0.001, 0.01, 0.05]
              }

In [180]:
grid_search2 = HalvingGridSearchCV(pipeline2, param_grid2, cv=5, n_jobs=-1, verbose=3,scoring ='neg_root_mean_squared_error')
#grid_search2.fit(X_train, y_train)

These are predictions and RMSE of the best model and the corresponding parameters.

In [181]:
#prediction_NN=grid_search2.predict(X_test)
#grid_search2.best_params_
#grid_search2.best_score_

The final model is the following (the pipeline takes really long).

In [133]:
svr = SVR(C=0.1,gamma=0.25, kernel="rbf") 
knn = KNeighborsRegressor(n_neighbors=15)
dt = DecisionTreeRegressor(max_depth=50)
NN = MLPRegressor(activation="relu",hidden_layer_sizes=(20,10,5), alpha=0.01)

ensemble_best =StackingRegressor(estimators=[('svr', svr), ('knn', knn), ('lr', lr),  ('dt', dt)], final_estimator=NN)


And these are the corresponding predictions with the best model, on the preprocessed data.

In [130]:
preprocessing.fit(X_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/category_encoders/target_encoder.py:122: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/category_encoders/target_encoder.py:127: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."


ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('robustscaler',
                                                  RobustScaler()),
                                                 ('knnimputer', KNNImputer())]),
                                 Index(['HeartRate_Min', 'HeartRate_Max', 'HeartRate_Mean', 'SysBP_Min',
       'SysBP_Max', 'SysBP_Mean', 'DiasBP_Min', 'DiasBP_Max', 'DiasBP_Mean',
       'MeanBP_Min', 'MeanBP_Max', 'MeanBP_Mean', 'RespRate_Min',
       'RespRate_Max', 'RespRate_Mean', 'TempC_Min', 'T...
      dtype='object')),
                                ('cat',
                                 Pipeline(steps=[('leaveoneoutencoder',
                                                  LeaveOneOutEncoder()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 Index(['GENDER', 'ADMISSION_TYPE', 'INSURANCE', 'RELIGION', 'MARITAL_STATUS',
       'ETHNICITY', 'FIRST_CAREUNIT'],
      dtype='object')),
                                ('icd9',
                                 Pipeline(steps=[('targetencoder',
                                                  TargetEncoder()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['ICD9_diagnosis'])])

In [131]:
X_train_temp = preprocessing.transform(X_train)
columns_X = X_train.columns
X_train_temp = pd.DataFrame(X_train_temp,columns=columns_X)

X_test_temp = preprocessing.transform(X_test)
X_test_temp = pd.DataFrame(X_test_temp,columns=columns_X)

In [134]:
ensemble_best.fit(X_train_temp,y_train)

StackingRegressor(estimators=[('svr', SVR(C=0.1, gamma=0.25)),
                              ('knn', KNeighborsRegressor(n_neighbors=15)),
                              ('lr', LinearRegression()),
                              ('dt', DecisionTreeRegressor(max_depth=50))],
                  final_estimator=MLPRegressor(alpha=0.01,
                                               hidden_layer_sizes=(20, 10, 5)))

In [135]:
y_pred=ensemble_best.predict(X_test_temp)

In [141]:
prediction_submit_ensemble=pd.DataFrame({"icustay_id":icustay_id_test, "LOS": y_pred})
prediction_submit_ensemble.to_csv("prediction_submit_ensemble.csv", index=False)


## Conclusion

Overall, I found stacking a really interesting ensemble method. I think that this approach can be effective when the weak learners are diverse as it allows to combine their complementary strengths. Indeed, the predictions from the weak learners are combined using a meta algorithm (like a neural network) to make a final prediction.

This reasoning stems from the nature of bagging and boosting. Indeed, Bagging usually reduces the variance of the models while Boosting reduces the bias. Therefore, for Bagging algorithm it would makes sense to use low bias high variance weak learners and for Boosting it is better to use low variance high bias base models.

Hence, stacking can be an interesting method as we can include different models with different nature in terms of bias-variance tradeoff.